# Capitulo 5: Spark SQL and DataFrames: Spark SQL and DataFrames: Interacting with External Data Sources

Iremos leyendo y realizando los ejemplos del capitulo 5 del libro, complementandolo con unos actividades sobre los mismos ejemplos.

##### Siempre debemos iniciar una instancia SparkSession al principio. 

Por lo que antes de comenzar crearemos la SparkSession correspondiente:

In [19]:
import pyspark 
from pyspark.sql import SparkSession

# Creamos SparkSession
spark = (SparkSession
         .builder
         .appName("LibroSpark_cap5")
         .master("local")
         .enableHiveSupport()
         .getOrCreate())

En el capitulo 4 exploramos e interactuamos con las diferentes fuentes de datos y formatos que ofrece Spark, la api DF y SparkSQL. En este capitulo nos centraremos como SparkSQL permite:
- Usar UDFs (User Define Functions) para Apache Hive y Apache Spark
- Conectar con fuentes de datos externas JBDC y bbdd SQL como PostgreeSQL, MySQL, Tableau, MS SQL Server
- Trabajar con tipos de datos complejos, funciones de alto nivel y operadores relacionales comunes.
- Adicionalmente miraremos diferenicas entre lanzar querys de Spark con SparkSQL como la shell, el cliente beeline o Tableau

## User-Definded Functions

Las UDFs son funciones creadas o definidas por el propio usuario, sirviendo de sistema para definir metodos SQL para operar sobre las columnas de un DF o Tabla. Se usan para definir una nueva columna basada en una función que extiende el vocabilario de SparkSQL para transformas DFo DS. Las UDFs se operan por sesión y no se guardan en el metastore.

En cuanto a los Pros:
- Se pueden utilizar por otros usuarios
- No es necesario entender 100% su funcionamiento interno para su uso
- Operan por cada SparkSession y no se guardan en el metastore 

En cuanto a los contras:
- Su rendimiento es inferior en Python (Esto se resuelve usando Pandas/Vectorized UDFs) que en Scala o Java.
- Spark SQL no puede optimizarlas
- No garantiza el orden de evaluación de las subexpresiones/querys.

#### Spark SQL UDFs
Breve ejemplo de creacion de UDF de SparkSQL. Hay que tener en cuenta que las UDFs operan por sesion y no se guardan en el metastore:

In [20]:
from pyspark.sql.types import *

# Creamos funcion para elevar al cubo
def cubed(s):
    return s * s * s

# Registramos nuestra UDF
spark.udf.register("cubed", cubed, LongType())

<function __main__.cubed(s)>

In [21]:
# Generate temporary view de ejemplo
spark.range(1, 9).createOrReplaceTempView("udf_test")

# Ahora podemos usar nuestra UDF cubed() con SparkSQL 
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test ").show()

+---+--------+
| id|id_cubed|
+---+--------+
|  1|       1|
|  2|       8|
|  3|      27|
|  4|      64|
|  5|     125|
|  6|     216|
|  7|     343|
|  8|     512|
+---+--------+



### Speeding up and distributing PySpark UDFs with Pandas UDFs

Una de los problemas que prevalecian con el uso de UDFs en Pyspark era que su rendimiento es mas lento que en Scala. Esto se debe a que PySpark UDFs tienen que mover los datos entre la JVM y Python, lo que es caro en terminos de memoria.

Para resolver esto se utilizan las **Pandas UDFs o Vectorized UDFs**. Desde Spark 3.0 y Python 3.6 Pandas UDFs fue divido en 2 categorías:
- Pandas UDFs. 
- Pandas Function APIs. Permiten udar una funcion local de Python a un DF de Pyspark donde la entrada y la salida son instancia de pandas. En Spark 3.0 soportan las funciones grouped map, map, cogrouped map.

A diferencia de una función local, el uso de una UDF vectorizada **dará lugar a la ejecución de trabajos de Spark**; la función local anterior es una función de Pandas ejecutada sólo en el controlador de Spark.

Ejemplo de Pandas UDFs

In [22]:
# Importamos varias funciones pyspark SQL incluida pandas_udf
import pandas as pd
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Creamos funcion cubed para elevar numeros al cubo
def cubed(a: pd.Series) -> pd.Series:
    return a * a * a

# Creamos la UDF pandas/vectorizada para nuestra funcion cubed creada
# cubed_udf = pandas_udf(cubed, returnType=LongType())

# ImportError: PyArrow >= 0.15.1 must be installed; however, it was not found.

In [23]:
# Creamos DF Pandas Series
x = pd.Series([1, 2, 3])

# Usamos la funcion cubed
print(cubed(x))

0     1
1     8
2    27
dtype: int64


In [24]:
# Creamos Spark DF
df = spark.range(1, 4)

# Ejecutamos UDF pandas/vectorizada 
# df.select("id", cubed_udf(col("id"))).show()

## Querying with the Spark SQL Shell, Beeline, and Tableau

Hay varias formas de lanzar Querys en Spark como la Spark SQL Shell, el cliente Beeline, aplicaciones de reporting como Tableau o PowerBI, Jupyer notebook como es este caso,etc.

En este caso veremos un ejemplo en "Ejs_word_cap5" que muestra el uso de conectores JBDC para usar una RDBMS con MySQL
 


## Higher-Order Functions

Este tipo de funciones se utilizan para los tipos de datos complejos (que son abstracciones de tipos de datos ismples), tenemos la gentaja de que nos permita pensar en un porblema de manera tabular aun usando tipos de datos complejos.

Aparte de las funciones mencionadas anteriormente, existen funciones de alto nivel que toman funciones anonimas formadas por expresiones lambda como argumentos. Como por ejemplo, en SQL:

--SQL
transform(values, value -> lambda expression)

La función tranform() coje un array (values) y una función anónima (una expresión lambda) como entrada. La función crea de forma transparente un nuevo array aplicando la función anónima a cada elemento, y luego asignando el resultado al array de salida, este proceso es similar al enfoque UDF, pero más eficiente.

Probaremos algunas hihg order functions, en primer lugar creamos nuestro DF de ejepmlo t_c y una vista temporal tC

In [25]:
from pyspark.sql.types import *

# Creamos el schema primero (una unica columna celsius de tipo array int)
schema = StructType([StructField("celsius", ArrayType(IntegerType()))])
# Creamos lista de datos (dos arrays de numeros enteros)
t_list = [[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]
# Creamos DF
t_c = spark.createDataFrame(t_list, schema)
# Creamos vista temporal a través del DF
t_c.createOrReplaceTempView("tC")

# lo mostramos para verificar que está correcto
t_c.show()

+--------------------+
|             celsius|
+--------------------+
|[35, 36, 32, 30, ...|
|[31, 32, 34, 55, 56]|
+--------------------+



#### *Función transform()*
Esta funcion produce un array aplicando una función a cada elemento de la matriz/array (similar a la función map)

In [26]:
# Transformamos nuestra columna de temperatura celsius a farenheit con nuestra vista temporal tC para hacerlo en SQLpuro
# Creamos nueva columna farenheit utilizando la función transform
# Para ello usamos la funcion transform aplicando la formula de pasar de grados celsius a farenheit

spark.sql(""" SELECT celsius, transform(celsius, t -> ((t * 9) div 5) + 32) as fahrenheit
              FROM tC """).show()

+--------------------+--------------------+
|             celsius|          fahrenheit|
+--------------------+--------------------+
|[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
|[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+



#### *Función filter()*
Esta funcion produce una matriz que contiene sólo en los elementos de la matriz de entrada para los que se cumole una condicion booleana que es verdadera.

In [27]:
# Utilizamos filter aplicando un filtro a t que coge solo la temperaturas mayores que 38
spark.sql(""" SELECT celsius, filter(celsius, t -> t > 38) as high
              FROM tC """).show()

+--------------------+--------+
|             celsius|    high|
+--------------------+--------+
|[35, 36, 32, 30, ...|[40, 42]|
|[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+



#### *Función exists()*
Esta funcion devuelve true si la función booleana es válida para cualquier elemento de la matriz de entrada:

In [28]:
# Utilizamos exists para ver si hay tempraturas mayores de 30 en cada array/matriz en una nueva columna thresold
spark.sql(""" SELECT celsius, exists(celsius, t -> t = 38) as threshold
              FROM tC """).show()

# como podemos observar, para el primer array si existen t>38 mientras que para el segundo array no

+--------------------+---------+
|             celsius|threshold|
+--------------------+---------+
|[35, 36, 32, 30, ...|     true|
|[31, 32, 34, 55, 56]|    false|
+--------------------+---------+



#### *Función reduce()*
Esta funcion reduce los elementos del array a un unico valor fusionando los elementos y aplicando una función a los mismos
(fusionando los elementos en un buffer B usando la función<B, T, B> y aplicando una función unción<B, R> en el buffer final)


In [29]:
# Utilizamos reduce para aplicar la media de todos los valores del array a la vez que los pasamos a farenheit con una nueva columna avgFarenheit
spark.sql(""" SELECT celsius, reduce(celsius, 
                     0, (t, acc) -> t + acc,acc -> (acc div size(celsius) * 9 div 5) + 32) as avgFahrenheit
                     FROM tC """).show()

AnalysisException: Undefined function: 'reduce'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 17

## Common DataFrames and Spark SQL Operations

Parte del poder de SparkSQL viene de el amplio rango de operacione con DF que soporta, podemos engobar estas operaciones o funciones más comunes en:
- Funciones de agregado
- Funciones de colección
- Funciones para datos de tipo fecha
- Funcioines matemáticas
- Funciones varias (miscelanea)
- Funciones no agregadas
- Funciones de agregado
- Funciones para datos de tipo string
- Funciones UDF
- Funciones de ventana

Se puede consultar la lista completa en la documentación de SparkSQL
https://spark.apache.org/docs/latest/api/sql/index.html

En esta sección nos en estos tipos de funciones y operaciones relacionales:
- Uniones y Joins
- Ventana
- Modificaciones

Para llevar acabo estas operaciones y ejemplos, prepararemos un DF en el siguiente codigo de ejemplo:

1. Importe dos archivos y cree dos DataFrames, uno para la **información del aeropuerto (airportsna)** y otro para los **retrasos de los vuelos de EEUU (departureDelays)**.

2. Utilizando expr(), **convierta las columnas de retrasos y distancias de STRING a INT.**

3. Cree una **tabla más pequeña, foo**, en la que podamos centrarnos para nuestros ejemplos de demostración; sólo contiene **información sobre tres vuelos con origen en Seattle (SEA) y destino en San Francisco (SFO) para un pequeño intervalo de tiempo.**




1. Importe dos archivos y cree dos DataFrames, uno para la información del aeropuerto (airportsna) y otro para los retrasos de los vuelos de EEUU (departureDelays).

In [31]:
from pyspark.sql.functions import expr
    
# Marcamos las rutas de los 2 datasets (departuredelays.csv y airport-codes-na.txt)
tripdelaysFilePath = "./Datasets/departuredelays.csv"
airportsnaFilePath = "./Datasets/airport-codes-na.txt"

# Creamos airportsna a partir de airport-codes-na y creamos una vista temporal
airportsna = (spark.read.format("csv")
                   .options(header="true", inferSchema = "true", sep="\t")
                   .load(airportsnaFilePath))

airportsna.createOrReplaceTempView("airports_na")

# Creamos departureDelays a partir de departuredelays.csv y creamos una vista temporal
departureDelays = (spark.read.format("csv")
                         .options(header="true")
                         .load(tripdelaysFilePath))

# verificamos que todo se ha creado bien
airportsna.show(5)
departureDelays.show(5)



+----------+-----+-------+----+
|      City|State|Country|IATA|
+----------+-----+-------+----+
|Abbotsford|   BC| Canada| YXX|
|  Aberdeen|   SD|    USA| ABR|
|   Abilene|   TX|    USA| ABI|
|     Akron|   OH|    USA| CAK|
|   Alamosa|   CO|    USA| ALS|
+----------+-----+-------+----+
only showing top 5 rows

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 5 rows



2. Utilizando expr(), convierta las columnas de retrasos y distancias de STRING a INT.

In [32]:
# Modificamos las columnas delay y distance transformadolas a datatype int en el DF departureDelays
 # para ello utilizamos expr
departureDelays = (departureDelays.withColumn("delay", expr("CAST(delay as INT) as delay"))
                                  .withColumn("distance", expr("CAST(distance as INT) as distance")))

# Creamos su correspondiente vista temporal
departureDelays.createOrReplaceTempView("departureDelays")



3. Cree una tabla más pequeña, foo, en la que podamos centrarnos para nuestros ejemplos de demostración; sólo contiene información sobre tres vuelos con origen en Seattle (SEA) y destino en San Francisco (SFO) para un pequeño intervalo de tiempo.

In [33]:
# Create tabla temporal foo
# Esta subtabla coniene la información de los vuelos con origen en seattle y san francisco para un pequeño intervalo de tiempo
foo = (departureDelays.filter(expr("""origin == 'SEA' and destination == 'SFO' and
                                      date like '01010%' and delay > 0""")))

foo.createOrReplaceTempView("foo")

El DF departureDelays contiene más de 1.3M de vuelos mientras que el DF foo contiene solo 3 vuelos de SEA y SFO para un rango específico de tiempo. Podemos ver todo esto en el siguiente ejemplo en el que verificamos que todo ha sido creado correctamente:

In [34]:
# Utilizamos spark.sql para hacer un select rapido en sqlpuro para ver que el contenido está bien
spark.sql("SELECT * FROM airports_na LIMIT 10").show()
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()
spark.sql("SELECT * FROM foo").show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        AT

Una vez tenemos todos nuestros datos limpios y preparados, realizamos unos ejemplos de operaciones/funciones relacionales:
- union y joins
- ventana
- modificaciones

#### Union
Esta operación une dos DF diretentes con un el mismo schema. Para ello utilizamos el metodo union()

In [29]:
# Unimos los DF departureDelays y foo 
bar = departureDelays.union(foo)

# Creamos una tabla temporal bar
bar.createOrReplaceTempView("bar")

# Mostramos la union filtrando or SEA and SFO en un rango de tiempo específico
bar.filter(expr("""origin == 'SEA' AND destination == 'SFO' AND date LIKE '01010%' AND delay > 0""")).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



#### Joins
Esta operación join une dos DF diretentes por COLUMNAS con un el mismo schema. Para ello utiliza el metodo join(). Por defecto, SparkSQL utiliza el join de tipo "inner join", aunque puedes cambiar a otros tipos de joins como:
- being inner 
- cross
- outer 
- full 
- full_outer 
- left 
- left_outer 
- right 
- right_outer 
- left_semi 
- oleft_anti

En el siguiente ejemplo realizamos un iner join uniendo los DF airportsna y foo, en lenguaje DF y SQLpuro

In [35]:
# Join en lenguaje DF por las columnas IATA y origin
( foo.join(airportsna, airportsna.IATA == foo.origin)
    .select("City", "State", "date", "delay", "distance", "destination").show() )
    
# Join en lenguaje SQLpuro con spark.sql por las columnas IATA y origin
spark.sql(""" SELECT a.City, a.State, f.date, f.delay, f.distance, f.destination
              FROM foo f
              JOIN airports_na a
              ON a.IATA = f.origin """).show()

# La query consiste en mostrar las columnas fecha, el retraso, la distancia 
# y el destino del DataFrame de foo unida a la información de la ciudad y el estado de el DataFrame de los aeropuertos

+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+

+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+



#### Windowing

Una función de ventana utiliza los valores de las filas de una ventana (conjunto de filas de entrada) para devolver un conjunto de valores en otra fila. Gracias a ellas es poosible operar en un grupo de filas mientras se devuelve un unico valor para cada fila de entrada. 

En el siguiente ejemlo mostramos como utilizar la funcion de ventana dense_rank(). 

En primer lugar creamos una nueva tabla temp departureDelaysWindow creando la columna TotalDelays (calculada son sum(Delay)) experimentados por los vuelos de origen SEA, SFO y JFK que van a un conjuto de destinos (WHERE) que se indica en la siguiente query

In [37]:
# Creamos tabla departureDelaysWindow como CTA haciendo un seelect de la tabla departureDelaysWindow
# creando la columna TotalDelays 
spark.sql ( """DROP TABLE IF EXISTS departureDelaysWindow""" )

spark.sql ( """CREATE TABLE departureDelaysWindow AS
            SELECT origin, destination, SUM(delay) AS TotalDelays
            FROM departureDelays
            WHERE origin IN ('SEA', 'SFO', 'JFK') AND destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL')
            GROUP BY origin, destination""" )

spark.sql ( """SELECT * FROM departureDelaysWindow""" )

DataFrame[]

Ahora queremos buscar los 3 destinos que tienen más retrasos, se puede lograr haciendo 3 consultas diferentes y luego uniendo los resultados

In [ ]:

spark.sql (  """SELECT origin, destination, SUM(TotalDelays) AS TotalDelays
             FROM departureDelaysWindow
             WHERE origin = '[ORIGIN]'
             GROUP BY origin, destination
             ORDER BY SUM(TotalDelays) DESC
             LIMIT 3""" )

O puedes hacerlo utilizando la función dense_rank para desarrollar este calculo. 
Así, con la fucnión dense_rank podemos ver un ranking de los 3 vuelos con más retraso para cada orgien.

Es importante mencionar que cada función de ventana necesita caber en un solo executor y estará compuesta por una sola partición durante la ejecución. Por lo que, tenemos que asegurarnos de que sus querys no sean ilimitadas, es decir, limitamos cada tamaño de cada ventana.

In [41]:
## utilizamos funcion dense_rank
spark.sql(""" SELECT origin, destination, TotalDelays, rank
              FROM ( 
                SELECT origin, 
                       destination, 
                       TotalDelays, 
                       dense_rank() 
                       OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank
                     FROM departureDelaysWindow) t
              WHERE rank <= 3
""").show()

#### Modifications

Otra operación comun es desarrollar modificaciones en el DF. Como el DF es inmutable, realizas modificaciones en nuevo DF o sobreescribiendo el mismo. 

Utilizaremos el DF foo para estos ejemplos y llevaremos a cabo
- Añadir columnas
- Borrar columnas
- Renombrar columnas
- Pivotar


In [42]:
foo.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



##### Añadir columnas

Para añadir columnas utilizamos el metodo withColumn(). En el ejemplo crearemos el DF foo2 en el que añadimos la columna status definida con una sentencia case utilizando la función expr()

El DF foo2 contiene el contenido de foo y la columna adicional status. 
Esta columna utiliza la función expre para poner la sentencia case de que cuando la columna delay sea inferior a 10 poner el status "On-time", en caso contrario poner "Delayed"

In [47]:
from pyspark.sql.functions import *
foo2 = ( foo.withColumn( "status",
                        expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END") 
                       ) ).show()

+--------+-----+--------+------+-----------+-------+
|    date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
|01010710|   31|     590|   SEA|        SFO|Delayed|
|01010955|  104|     590|   SEA|        SFO|Delayed|
|01010730|    5|     590|   SEA|        SFO|On-time|
+--------+-----+--------+------+-----------+-------+



##### Borrar columnas

Para borrar columnas utlizamos el metodo drop(). En el ejemplo elminamos la columa delay ya que tenemos la columna status del ejemplo anterior que nos da la información de si un vuelo se retrasó o no de manera mas legible.

Creamos un nuevo DF foo3 con todos estos cambios

In [63]:
foo3 = foo2.drop("delay").show()

AttributeError: 'NoneType' object has no attribute 'drop'

##### Renombrar columnas

Puedes renombrar columnas con el metodo rename() seguido del metodo withColumn().

Creamos un nuevo DF foo4 con estos cambios

In [61]:
foo4 = foo2.withColumnRenamed("status", "flight_status")
foo4.show()

AttributeError: 'NoneType' object has no attribute 'withColumnRenamed'

##### Pivotar

Pivotar significa cambiar las columnas por las filas. Veamoslo en el siguiente ejemplo:

Pivotar permite colocar nombres en la columna del mes (en lugar de 1 y 2 puede mostrar enero y febrero, respectivamente), así como realizar cálculos agregados (en este caso promedio y máximo) sobre los retrasos por destino y mes

In [65]:
# vemos en este selct que los meses se muestran en numeros (importante pasar la columna a srting)
spark.sql ( """SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
            FROM departureDelays
            WHERE origin = 'SEA'""" ).show()

+-----------+-----+-----+
|destination|month|delay|
+-----------+-----+-----+
|        ORD|    1|   92|
|        JFK|    1|   -7|
|        DFW|    1|   -5|
|        MIA|    1|   -3|
|        DFW|    1|   -3|
|        DFW|    1|    1|
|        ORD|    1|  -10|
|        DFW|    1|   -6|
|        DFW|    1|   -2|
|        ORD|    1|   -3|
|        ORD|    1|    0|
|        DFW|    1|   23|
|        DFW|    1|   36|
|        ORD|    1|  298|
|        JFK|    1|    4|
|        DFW|    1|    0|
|        MIA|    1|    2|
|        DFW|    1|    0|
|        DFW|    1|    0|
|        ORD|    1|   83|
+-----------+-----+-----+
only showing top 20 rows



In [67]:
spark.sql (""" SELECT * FROM (
               SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
                 FROM departureDelays WHERE origin = 'SEA'
           )
               PIVOT (
                CAST(AVG(delay) AS DECIMAL(4, 2)) AS AvgDelay, MAX(delay) AS MaxDelay
                FOR month IN (1 JAN, 2 FEB)
           )
               ORDER BY destination""").show()

+-----------+------------+------------+------------+------------+
|destination|JAN_AvgDelay|JAN_MaxDelay|FEB_AvgDelay|FEB_MaxDelay|
+-----------+------------+------------+------------+------------+
|        ABQ|       19.86|         316|       11.42|          69|
|        ANC|        4.44|         149|        7.90|         141|
|        ATL|       11.98|         397|        7.73|         145|
|        AUS|        3.48|          50|       -0.21|          18|
|        BOS|        7.84|         110|       14.58|         152|
|        BUR|       -2.03|          56|       -1.89|          78|
|        CLE|       16.00|          27|        null|        null|
|        CLT|        2.53|          41|       12.96|         228|
|        COS|        5.32|          82|       12.18|         203|
|        CVG|       -0.50|           4|        null|        null|
|        DCA|       -1.15|          50|        0.07|          34|
|        DEN|       13.13|         425|       12.95|         625|
|        D